In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Ilya Jeff\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import seaborn as sns
import glob
import torch
import os

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
mpl.rcParams['agg.path.chunksize'] = 10000

#from matplotlib import rc
#rc('text', usetex=True)
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'


In [41]:
rerun_rounds = ['round1', 'round2', 'round3']
def get_avg_results(path_string):
  """
  For experiments with multiple random training runs, collects all
  training runs and averages their results
  """
  result_list = []
  for rerun_string in rerun_rounds:
    completed_path = path_string.format(rerun_string)
    if os.path.exists(completed_path):
      result_list.append(float(open(completed_path).read()))
  return sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0
  #print(result_list)
  #return np.std(result_list) if len(result_list) > 0 else -1 #sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0

In [44]:
a = torch.tensor([get_avg_results('reports/layer{}-0.yaml.results/dev.v_entropy'.format(i)) for i in range(1,13)])
b = torch.tensor([float(open('reports/layer{}-0.yaml.results/dev.v_entropy'.format(i)).read().strip()) for i in range(1,13)])
c = get_avg_results('reports/layer0.yaml.results/dev.v_entropy')
print(c)

#for i in range(1,13):
#   print(i)

tensor([4.1029, 4.0838, 4.0796, 4.0765, 4.0777, 4.0805, 4.0839, 4.0834, 4.0885,
        4.0911, 4.0936, 4.1009])


In [42]:
def plot_layerwise_zero_baseline():
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""

    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/layer{}-0.yaml.results/dev.v_entropy'.format(i)) for i in range(1,13)])
    task_768_embedding_layer_ventropy = get_avg_results('reports/layer0.yaml.results/dev.v_entropy')
    #task_768_twolayer_ventropy = torch.tensor([float(open('reports/layer{}-0.yaml.results/dev.v_entropy'.format(i)).read().strip()) for i in range(1,13)])
    #task_768_embedding_layer_ventropy = float(open('reports/layer0.yaml.results/dev.v_entropy').read().strip())
    x_range = list(range(1,13))
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')
    plt.title('roberta dep rel')
    plt.ylabel('Bits')
    plt.xlabel('Model layer')
    plt.legend()
    plt.tight_layout()
    plt.savefig('roberta dep rel', dpi=200)
    plt.clf()

In [43]:
plot_layerwise_zero_baseline()